In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
from googletrans import Translator
import re
from os import listdir
import shutil
import csv
from  collections import defaultdict
from nltk.corpus import stopwords

In [2]:
translator = Translator()

In [3]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [4]:
def isLatin(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True



In [5]:
def cleanTweet(t):
    
    t=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", t).split())
    
    e=0
    for indice, letter in enumerate(t):
        
        if letter.isupper():
            if indice!=0:
                t=t[:indice+e] + ' ' + t[indice+e:]
                e+=1
    
    
    t=split_numbers(t)
    t=clean_tweet_ngrams(t)
    return t

In [6]:
# cleaning 2: define (includes stopwords + customised stop words)
stop_it = stopwords.words('english') + ['rt','RT','via','️','❗','…',"0","1","2","3","4","5","6","7","8","9"]
def clean_tweet_ngrams(tweet,stemming=False):
     '''
     Utility function to clean the text in a tweet by trasnforming to lower case,
     stemming and removing stopwords
     '''
     tweet = tweet.lower()
     tweet = tweet.split()
     if stemming==False:
         tweet = [word for word in tweet if not word in stop_it]
     else:
         ps = PorterStemmer()
         tweet = [ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
     tweet = ' '.join(tweet)
     return tweet

In [7]:
def translateTweet(t):
    t=re.sub('#', '', t)
    traducido=translator.translate(t)
    #print(t, traducido.text)
    return traducido.text

In [8]:
def split_numbers(t):
    t=re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", t).strip()
    return t

In [20]:
def find_known_themes(data_recibida,date):
    #print('the data has ' + str(data.shape[1]) + ' columns' )
    for column in data_recibida:
        
        g=1
        if isLatin(column)== False : 
            tweet=translateTweet(column)
            tweet=cleanTweet(tweet)
            if tweet=='':tweet=column
            #print('translated '+column+' to '+tweet)
        else: tweet=column
        column_joint=tweet.replace(' ','').lower()
        if column_joint[0] != '#': column_hash='#'+column_joint
        else: column_hash=column_joint
        #print('         '+column_hash)
        if column_hash in dictionary_topics.keys():       
            known_topics.append([dictionary_topics[column_hash],df.loc['tweet_volume',column],df.loc['duration',column],date])
            known_events.append([dictionary_events[column_hash],df.loc['tweet_volume',column],df.loc['duration',column],date])
            data_recibida = data_recibida.drop(columns=[column])
        

        elif 'trump' in column_joint:
            known_topics.append(['politics & economy',df.loc['tweet_volume',column],df.loc['duration',column],date])
            known_events.append(['Trump',df.loc['tweet_volume',column],df.loc['duration',column],date])
            data_recibida = data_recibida.drop(columns=[column])
            
        elif 'corona' in column_joint or 'covid' in column_joint or 'lockdown' in column_joint:
            known_topics.append(['health',df.loc['tweet_volume',column],df.loc['duration',column],date])
            known_events.append(['COVID',df.loc['tweet_volume',column],df.loc['duration',column],date])
            data_recibida = data_recibida.drop(columns=[column])
        else: g=0
        #if g==1: print('>>>>>>>>change at tweet '+column )
    #print('the data returns with ' + str(data.shape[1]) + ' columns' )
    return data_recibida

In [70]:
def add_equals(dataframe):
    dataframe = dataframe.groupby(['tweet', 'date']).agg({'weight': ['sum'],'duration':['sum']})
    dataframe.columns = [ 'weight','duration']
    
    dataframe = dataframe.reset_index()
    dataframe_sorted=dataframe.sort_values(['date'])
    return dataframe_sorted

In [11]:
dictionary_events=defaultdict(list)
dictionary_topics=defaultdict(list)
with open('./dictionary.csv') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for indexr, row in enumerate(spamreader):
        if indexr==0:continue
        for indexc,column in enumerate(row):
            if indexc==2:
                column.replace('#',' ')
                column.replace('\n\n','\n')
                terms=[t for t in column.split('\n')]
                    
                    
                '''else:
                    terms=[t for t in column.split(' ')]'''
                for term in terms:
                    if term!='':
                        term=term.replace(' ','')
                        term=term.replace('\n','').lower()
                        if term[0]!='#':term='#'+term
                        dictionary_events[term]= row[1] 
                        dictionary_topics[term]= row[0] 
            else:continue
    csvfile.close()
dictionary_topics

defaultdict(list,
            {'#eurovision': 'culture & sports',
             '#esc': 'culture & sports',
             '#junioreurovision': 'culture & sports',
             '#eurovisionsongcontest': 'culture & sports',
             '#pseudoscience': 'culture & sports',
             '#fakenews': 'culture & sports',
             '#eurocup': 'culture & sports',
             '#euro2020': 'culture & sports',
             '#nationsleague': 'culture & sports',
             '#europeanqualifiers': 'culture & sports',
             '#copaamerica': 'culture & sports',
             '#conmebol': 'culture & sports',
             '#copaamérica': 'culture & sports',
             '#uefa': 'culture & sports',
             '#bundesliga': 'culture & sports',
             '#football': 'festivities & cults',
             '#soccer': 'culture & sports',
             '#nfl': 'festivities & cults',
             '#sports': 'festivities & cults',
             '#futbol': 'culture & sports',
             '#fifa': '

In [12]:
path_json= './json/no_procesados/'
path_json_procesados= './json/procesados'
path_csv='./csv/'
numero_a_procesar=200


In [13]:
e=0
known_topics=[['tweet','weight','duration','date']]
known_events=[['tweet','weight','duration','date']]
for f in sorted(listdir(path_json)):
    
    if f.endswith('.json'):
        print(f)
        print(e)
        e+=1
        if e>numero_a_procesar:break
        date=f[2:10]
        name_csv=path_csv+date.replace('-','')+'.csv'
        data =pd.read_json(path_json+f)
        print(data.shape)
        df=pd.DataFrame(data)
        #print (df)
        df=find_known_themes(df,date)
        #print(known_topics)
        unido=[]
        for column in df:
            if df.loc['tweet_volume',column]!=-1:
                parcial=[]
                if isLatin(column):
                    tweet_limpio=cleanTweet(column)
                    tweet_traducido=translateTweet(tweet_limpio)

                    parcial.append(tweet_traducido)
                    parcial.append(df.loc['tweet_volume',column])
                    parcial.append(df.loc['duration',column])
                    unido.append(parcial)

                else:
                    tweet_traducido=translateTweet(column)
                    tweet_limpio=cleanTweet(tweet_traducido)
                    #print('++++++2nda ronda '+column +' to' + tweet_limpio)
                    if isLatin(tweet_limpio):
                        parcial.append(tweet_limpio)
                        parcial.append(df.loc['tweet_volume',column])
                        parcial.append(df.loc['duration',column])
                        unido.append(parcial)

        dataf=pd.DataFrame(unido,columns=['tweet','weight','duration'])
        dataf.dropna(subset=['tweet'], inplace=True)
        df_sorted=dataf.sort_values(by=['weight'],ascending=False).reset_index(drop=True)
        df_sorted.to_csv(name_csv, index = False)
        
        shutil.move(path_json+f, path_json_procesados)


2020-01-02_1.json
0
(2, 104)
2020-01-03_1.json
1
(2, 488)
2020-01-04_1.json
2
(2, 498)
2020-01-05_1.json
3
(2, 523)
2020-01-06_1.json
4
(2, 484)
2020-01-07_1.json
5
(2, 366)
2020-01-08_1.json
6
(2, 476)
2020-01-09_1.json
7
(2, 503)
2020-01-10_1.json
8
(2, 479)
2020-01-11_1.json
9
(2, 490)
2020-01-12_1.json
10
(2, 519)
2020-01-13_1.json
11
(2, 482)
2020-01-14_1.json
12
(2, 481)
2020-01-15_1.json
13
(2, 478)
2020-01-16_1.json
14
(2, 493)
2020-01-17_1.json
15
(2, 476)
2020-01-18_1.json
16
(2, 506)
2020-01-19_1.json
17
(2, 610)
2020-01-20_1.json
18
(2, 486)
2020-01-21_1.json
19
(2, 488)
2020-01-22_1.json
20
(2, 478)
2020-01-23_1.json
21
(2, 497)
2020-01-24_1.json
22
(2, 508)
2020-01-25_1.json
23
(2, 516)
2020-01-26_1.json
24
(2, 538)
2020-01-27_1.json
25
(2, 452)
2020-01-28_1.json
26
(2, 497)
2020-01-29_1.json
27
(2, 488)
2020-01-30_1.json
28
(2, 481)
2020-01-31_1.json
29
(2, 537)
2020-02-01_1.json
30
(2, 532)
2020-02-02_1.json
31
(2, 569)
2020-02-03_1.json
32
(2, 458)
2020-02-04_1.json
33

In [14]:
df_known_topics=pd.DataFrame(known_topics[1:],columns=list(known_topics[0]))
df_known_events=pd.DataFrame(known_events[1:],columns=list(known_events[0]))

In [15]:
df_topics_ordered=add_equals(df_known_topics)
df_events_ordered=add_equals(df_known_events)

In [16]:
lista_topics=df_topics_ordered['tweet'].unique().tolist()
lista_events=df_events_ordered['tweet'].unique().tolist()
dates = pd.date_range('2020-01-01', '2020-06-30').tolist()
dates=[d.strftime("%-y-%m-%d") for d in dates]

In [64]:
#la lista de los topics segun el excel del drive
lista_topics_excel=['festivities & cults',
                    'violence & war',
                    'politics & economy',
                    'health','mobility & borders',
                    'environment',
                    'science & technology',
                    'social issues',
                    'culture & sports',
                    'social media bullshit']
#CREAMOS EMPTY DATAFRAMES CON LA ESTRUCTURA QUE QUEREMOS PARA LLENARLOS
df_topics_durations = pd.DataFrame(0, index=lista_topics_excel, columns=dates)
df_topics_durations.sort_index(axis = 0, inplace=True)
df_topics_weights = pd.DataFrame(0, index=lista_topics_excel, columns=dates)
df_topics_weights .sort_index(axis = 0, inplace=True)
df_events_durations = pd.DataFrame(0, index=lista_events, columns=dates)
df_events_weights = pd.DataFrame(0, index=lista_events, columns=dates)

In [65]:
df_topics_weights

,20-01-01,20-01-02,20-01-03,20-01-04,20-01-05,20-01-06,20-01-07,20-01-08,20-01-09,20-01-10,...,20-06-21,20-06-22,20-06-23,20-06-24,20-06-25,20-06-26,20-06-27,20-06-28,20-06-29,20-06-30
culture & sports,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
environment,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
festivities & cults,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
health,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mobility & borders,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
politics & economy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
science & technology,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
social issues,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
social media bullshit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
violence & war,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
for index, row in df_known_topics.iterrows():  
    df_topics_weights[row['date']][row['tweet']] =  row['weight']
    df_topics_durations[row['date']][row['tweet']] =  row['duration']
for index, row in df_known_events.iterrows():
    df_events_weights[row['date']][row['tweet']] =  row['weight']
    df_events_durations[row['date']][row['tweet']] =  row['duration']

In [68]:
df_topics_weights


,20-01-01,20-01-02,20-01-03,20-01-04,20-01-05,20-01-06,20-01-07,20-01-08,20-01-09,20-01-10,...,20-06-21,20-06-22,20-06-23,20-06-24,20-06-25,20-06-26,20-06-27,20-06-28,20-06-29,20-06-30
culture & sports,0,0,0,-1,0,0,0,0,210710,230375,...,0,0,0,0,0,0,0,0,0,0
environment,0,0,21998,96287,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
festivities & cults,0,0,0,19232,93687,23959,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
health,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mobility & borders,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
politics & economy,0,0,11771,-1,0,0,0,-1,84828,21010,...,0,0,0,0,0,0,0,0,0,0
science & technology,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
social issues,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
social media bullshit,0,73838,47573,0,23131,16557,63766,66354,60132,-1,...,0,0,0,0,0,0,0,0,0,0
violence & war,0,0,146287,0,0,0,21870,255053,44858,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
df_topics_weights.to_csv('./200706_known_topics_weight.csv')

"""df_topics_durations.to_csv('./200706_known_topics_duration.csv')

df_events_weights.to_csv('./200706_known_events_weight.csv')
df_events_durations.to_csv('./200706_known_events_duration.csv')
"""

"df_topics_durations.to_csv('./200706_known_topics_duration.csv')\n\ndf_events_weights.to_csv('./200706_known_events_weight.csv')\ndf_events_durations.to_csv('./200706_known_events_duration.csv')\n"